In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,8.81,89,94,3.76,NZ,1686525984
1,1,port-aux-francais,-49.3500,70.2167,5.19,92,100,18.19,TF,1686525984
2,2,waitangi,-43.9535,-176.5597,13.79,93,100,5.85,NZ,1686525942
3,3,puerto deseado,-47.7503,-65.8938,4.92,65,40,13.72,AR,1686525984
4,4,isafjordur,66.0755,-23.1240,7.85,80,33,7.14,IS,1686525985


In [25]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,bethel,41.3712,-73.4140,25.20,60,0,4.12,US,1686526008
122,122,fallon,39.4735,-118.7774,24.95,40,0,2.57,US,1686526009
177,177,nushki,29.5542,66.0215,25.84,20,0,1.11,PK,1686526020
191,191,ha'il,27.5219,41.6907,26.01,25,0,0.00,SA,1686526022
201,201,colonia,50.9333,6.9500,21.59,48,0,3.60,DE,1686525999
207,207,oromocto,45.8351,-66.4792,21.96,43,0,2.06,CA,1686526027
246,246,mao,39.8885,4.2658,21.31,94,0,3.09,ES,1686526036
299,299,la foa,-21.7108,165.8276,22.13,53,0,1.82,NC,1686526047
351,351,xianyang,34.3378,108.7026,22.58,73,0,4.00,CN,1686526134
408,408,caribou,46.8606,-68.0120,22.92,48,0,2.06,US,1686526149


In [28]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
115,bethel,US,41.3712,-73.4140,60,
122,fallon,US,39.4735,-118.7774,40,
177,nushki,PK,29.5542,66.0215,20,
191,ha'il,SA,27.5219,41.6907,25,
201,colonia,DE,50.9333,6.9500,48,
207,oromocto,CA,45.8351,-66.4792,43,
246,mao,ES,39.8885,4.2658,94,
299,la foa,NC,-21.7108,165.8276,53,
351,xianyang,CN,34.3378,108.7026,73,
408,caribou,US,46.8606,-68.0120,48,


In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: No hotel found
fallon - nearest hotel: No hotel found
nushki - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
oromocto - nearest hotel: No hotel found
mao - nearest hotel: No hotel found
la foa - nearest hotel: No hotel found
xianyang - nearest hotel: No hotel found
caribou - nearest hotel: No hotel found
uruzgan - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
beira - nearest hotel: No hotel found
bismarck - nearest hotel: No hotel found
big bend - nearest hotel: No hotel found
worland - nearest hotel: No hotel found
halabjah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
115,bethel,US,41.3712,-73.4140,60,No hotel found
122,fallon,US,39.4735,-118.7774,40,No hotel found
177,nushki,PK,29.5542,66.0215,20,No hotel found
191,ha'il,SA,27.5219,41.6907,25,No hotel found
201,colonia,DE,50.9333,6.9500,48,No hotel found
207,oromocto,CA,45.8351,-66.4792,43,No hotel found
246,mao,ES,39.8885,4.2658,94,No hotel found
299,la foa,NC,-21.7108,165.8276,53,No hotel found
351,xianyang,CN,34.3378,108.7026,73,No hotel found
408,caribou,US,46.8606,-68.0120,48,No hotel found


In [30]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)